In [ ]:
#!/usr/bin/env python3
"""

Simulation Prediction Market Main Executable

"""

import argparse
import pandas as pd
import numpy as np
from random import choices
from Agents.bayesian_agent import BayesianAgent
from Market_Maker.market_maker import MarketMaker

Argument parser, we could specify verbose mode, agent number, and max iteration number for the market.

In [ ]:
def parse_command_line():
    """
    argument parser
    :return: args
    """
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "-v", "--verbose", action="store_true", help="verbose showing simulation process"
    )
    parser.add_argument(
        "-n", "--agent_number", type=int, help="specify agent number"
    )
    parser.add_argument(
        "-i", "--max_iteration", type=int, help="specify maximum iteration"
    )
    args = parser.parse_args()
    return args

Import data from csv file.

In [ ]:
def import_data(path):
    """
    import real data from csv file
    :param: path: data path
    :return:
    """
    res = pd.read_csv(path)
    res = res.drop(columns=["ds"])
    return list(res["y"])

accept arguments and use inputs to assign value for agent number, market max iteration number. 

In [ ]:
def main():
    args = parse_command_line()
    agent_number = args.agent_number
    max_iteration = args.max_iteration

specify the file path for the data we use.

In [ ]:
    data_path = "./Data/example_retail_sales.csv"

specify prior range, num prior (the agent would randomly pick num_prior=10 data in the prior_range=20 (first 20 data from the whole dataset) to form his initial belief).

In [ ]:
    prior_range = 20
    num_prior = 10

 Number of observed sample is the datapoint number the agent would see before he enters the market (he would see the previous 10 datapoints that is just before his enter time).

In [ ]:
    num_observed_sample = 10

import data, and we set the prior data using the first prior_range=20 data. The rest data would be the actual data corresponding to the market state at each timepoint.

In [ ]:
    # import data
    data = import_data(data_path)
    prior = data[:prior_range]
    data = data[prior_range:]

initialize the agents' state. They would randomly pick 10 datapoints from the 20 in prior data to form their beliefs. We then append these agents to a list.

In [ ]:
    # initialize agents
    agents = []
    for i in range(agent_number):
        agent_prior = choices(prior, k=num_prior)
        agent = BayesianAgent(agent_prior)
        agents.append(agent)

initialize the market maker. We then use the Poisson distribution to set the enter time for each agent. We store their enter time in enter_times as a list.

In [ ]:
    # initialize market maker
    mk = MarketMaker()
    enter_times = np.random.poisson(2.3, max_iteration)
    enter_times = list(np.cumsum(enter_times) + num_observed_sample)

as long as the iteration does not exceed the max_iteration number we assigned in the above, the current agent would come into the market with the assigned time (we assigned it by Poisson distribution from above).  

In [ ]:

    # Pass in the data which the agents need to update their beliefs
    # The number of data points passed in to agent should be num_observed_sample
    for i in range(max_iteration):
        agent_idx = i % agent_number
        curr_agent = agents[agent_idx]
        enter_time = enter_times[i]

 Each agent would observe 10 datapoints that are just before their enter time and use these to form their current belief (updat_param).

In [ ]:
         observed_data = data[-num_observed_sample + enter_time:enter_time]
         curr_agent.update_param(observed_data)

They would decide the amount of securites they would buy or sell ($\delta_1,\delta_2$).

In [ ]:
        delta_1, delta_2 = curr_agent.calculate_shares_to_buy(mk.num_trade, mk.current_market_price)

Agent and market maker would update security amount accordingly (update_security for agent and update_param for market maker).

In [ ]:
        curr_agent.update_security(delta_1, delta_2)
        mk.update_param(delta_1, delta_2)

Print out the final market price as the aggregated knowledge.

In [ ]:

    print("enter time: {}".format(enter_times))
    print(mk.current_market_price)


execute the whole program by calling the main function.

In [ ]:
if __name__ == '__main__':
    main()